In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
gbm = lgb.LGBMRanker()
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import ndcg_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import scipy
from sklearn.model_selection import ParameterSampler
from sklearn.model_selection import cross_val_score
from itertools import product
from sklearn.model_selection import GroupShuffleSplit

In [140]:
# Specify the file name
file_name = 'df_OptionB.csv'

# Specify the full file path
file_path = r'/Users/frederike/Documents/Artificial inteligence (Master)/Data mining techniques/Data-mining-assignment-2/' + file_name

# Read the dataset using pandas
df = pd.read_csv(file_path)

In [141]:
df.columns

Index(['srch_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'click_bool', 'booking_bool', 'score',
       'avg_comp_rate', 'avg_comp_inv', 'abroad_bool', 'children_bool',
       'srch_query_affinity_score_low', 'srch_query_affinity_score_high',
       'starrating_diff_low', 'starrating_diff_high', 'usd_diff_low',
       'usd_diff_high'],
      dtype='object')

In [142]:
df = df.sort_values(by='srch_id')

In [143]:
group = list(df['srch_id'].value_counts())

In [144]:
# Drop target variable and save it in separate variable
y = df['score']
X = df.drop(['score'], axis=1)

# Perform group based split
gss = GroupShuffleSplit(test_size=0.3, n_splits=1, random_state=42)
train_idx, test_idx = next(gss.split(X, y, groups=X['srch_id']))

# Create train and test sets
X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

# Create group data
group_train = list(X_train['srch_id'].value_counts().sort_index())
group_test = list(X_test['srch_id'].value_counts().sort_index())

# Drop unwanted columns
X_train = X_train.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)
X_test = X_test.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)

In [145]:
print(len(X_train))
print(len(X_test))

3471078
1487269


In [123]:
gbm.fit(X_train, y_train, group=group_train,
        eval_set=[(X_train, y_train), (X_test, y_test)], eval_group=[group_train, group_test],
        eval_at=5, early_stopping_rounds=50)

/Users/frederike/Library/Python/3.9/lib/python/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's ndcg@5: 0.306939	valid_1's ndcg@5: 0.306312
[2]	training's ndcg@5: 0.332153	valid_1's ndcg@5: 0.33042
[3]	training's ndcg@5: 0.340333	valid_1's ndcg@5: 0.337782
[4]	training's ndcg@5: 0.347333	valid_1's ndcg@5: 0.344098
[5]	training's ndcg@5: 0.350993	valid_1's ndcg@5: 0.347906
[6]	training's ndcg@5: 0.354228	valid_1's ndcg@5: 0.35126
[7]	training's ndcg@5: 0.355824	valid_1's ndcg@5: 0.352762
[8]	training's ndcg@5: 0.35779	valid_1's ndcg@5: 0.354551
[9]	training's ndcg@5: 0.358847	valid_1's ndcg@5: 0.355686
[10]	training's ndcg@5: 0.359955	valid_1's ndcg@5: 0.357139
[11]	training's ndcg@5: 0.361372	valid_1's ndcg@5: 0.357996
[12]	training's ndcg@5: 0.362214	valid_1's ndcg@5: 0.358813
[13]	training's ndcg@5: 0.362938	valid_1's ndcg@5: 0.35949
[14]	training's ndcg@5: 0.363681	valid_1's ndcg@5: 0.359786
[15]	training's ndcg@5: 0.364827	valid_1's ndcg@5: 0.360707
[16]	training's ndcg@5: 0.36532	valid_1's ndcg@5: 0.361375
[17]	training's ndcg@5: 0.365769	valid_1's ndcg@5: 0.3

LGBMRanker()

In [127]:
test_pred = gbm.predict(X_test)

In [128]:
results = gbm.evals_result_

In [129]:
results['valid_1']['ndcg@5'][-1]

0.3732928990200451

In [130]:
X_test["predicted_ranking"] = test_pred
X_test.sort_values("predicted_ranking", ascending=False)

,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_length_of_stay,srch_booking_window,...,avg_comp_inv,abroad_bool,children_bool,srch_query_affinity_score_low,srch_query_affinity_score_high,starrating_diff_low,starrating_diff_high,usd_diff_low,usd_diff_high,predicted_ranking
3676716,4,4.5,0,1.10,0.7598,4.91,71.00,1,1,0,...,0.0,0,0,0,0,0,0,0,0,2.907819
1692498,4,4.0,1,0.69,0.4056,4.85,69.00,1,2,1,...,0.0,0,0,0,0,0,0,0,0,2.885555
4401012,4,4.5,0,0.69,0.8814,5.38,89.00,1,3,17,...,0.0,1,1,0,0,0,0,0,0,2.848893
4386680,4,4.5,0,1.10,0.7177,5.06,79.00,1,2,1,...,0.0,0,0,0,0,0,0,0,0,2.848322
625808,5,4.0,1,0.69,0.6594,4.81,79.97,1,2,2,...,0.0,1,0,0,0,0,0,0,0,2.837351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2814621,0,5.0,0,0.00,0.0000,5.17,872.42,0,1,1,...,0.0,0,0,0,0,0,1,0,1,-2.822756
1109642,0,5.0,0,2.64,0.0000,4.91,630.00,0,1,1,...,0.0,0,0,1,0,0,0,0,0,-2.837037
1813388,5,4.5,0,5.34,0.0000,4.91,625.00,0,1,10,...,0.0,0,0,1,0,0,0,0,0,-2.843430
1813369,5,4.5,1,5.73,0.0000,4.91,795.00,0,1,10,...,0.0,0,0,1,0,0,0,0,0,-2.891384


In [131]:
print(gbm.get_params())

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': 'warn', 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}


In [132]:
file_name = 'df_testB.csv'
file_path = r'/Users/frederike/Documents/Artificial inteligence (Master)/Data mining techniques/Data-mining-assignment-2/' + file_name
df_test = pd.read_csv(file_path)

In [133]:
# Sort test set 
df_test = df_test.sort_values(by='srch_id')

# Creating group list to use in test set prediction
group = list(df_test['srch_id'].value_counts())

# Storing the removed columns in a separate DataFrame
df_test_columns_removed = df_test[['srch_id', 'prop_id']].copy()

# Create a new DataFrame by dropping the desired columns
df_test = df_test.drop(columns=['srch_id', 'prop_id'])

In [134]:
# Apply model to test set
pred = gbm.predict(df_test, group=group)

In [135]:
# Add predictions to test set
df_test["predicted_ranking"] = pred

# Adding the columns back
df_test['srch_id'] = df_test_columns_removed['srch_id']
df_test['prop_id'] = df_test_columns_removed['prop_id']

# Sort the dataframe by `srch_id` and `predicted_ranking` in descending order
test_sorted = df_test.sort_values(['srch_id', 'predicted_ranking'], ascending=[True, False])

In [136]:
result = test_sorted[['srch_id', 'prop_id']]
print(len(result))

4959183


In [137]:
result.to_csv('LambdaMart_result_unbalanced.csv', index=False)

### Hyperparametrisation 

In [148]:
#n_estimators= scipy.stats.randint(50,400)
max_depth= range(2,30, 2)
lambda_l1= np.arange(0,0.8,0.05)
lambda_l2=np.arange(0,0.8,0.05)
#subsample=np.arange(0.3,1.0,0.05)
#bagging_freq=scipy.stats.randint(1,100)
#colsample_bytree=np.arange(0.3,1.0,0.05)
num_leaves=range(20,100,10)
#boosting=['gbdt','dart']
#drop_rate=np.arange(0.1,0.8,0.1)
#skip_drop=np.arange(0,0.7,0.1)
learning_rate=[0.01,0.05,0.1,0.2]

#feature fraction 
#bagging fraction 
#min chill 
#bagging frequency 

params = {#'n_estimators':n_estimators,
          'max_depth':max_depth, 
          'is_unbalance':[True],
          #'subsample':subsample,
          #'colsample_bytree':colsample_bytree,
          'learning_rate':learning_rate,
          'num_leaves':num_leaves,
          #'boosting':boosting,
          #'lambda_l1':lambda_l1,
          #'lambda_l2':lambda_l2,
          #'bagging_freq':bagging_freq,
          #'drop_rate':drop_rate
          }


# We will store all parameter values in a list: 
param_vals = [val for val in params.values()]
# We will loop through all possible combinations from above / (*paramvals) returns all possible combinations of parameters
# then we return a list of fictionaries, where each one bind the initial keys from the params dictionary + each combination of parameters
param_combinations = [dict(zip(params.keys(), prod)) for prod in product(*param_vals)]

    

In [149]:
best_params = None
best_score = -1

for i in param_combinations:
    
    
    gbm = lgb.LGBMRanker(**i)
    gbm.fit(X_train, y_train, group=group_train,
        eval_set=[(X_train, y_train), (X_test, y_test)], 
        eval_group=[group_train, group_test],
        eval_at=5, early_stopping_rounds=10)
    
    
    results = gbm.evals_result_

    if(results['valid_1']['ndcg@5'][-1] > best_score):
        best_score = results['valid_1']['ndcg@5'][-1]


print(best_score)

/Users/frederike/Library/Python/3.9/lib/python/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's ndcg@5: 0.24169	valid_1's ndcg@5: 0.245005
[2]	training's ndcg@5: 0.26817	valid_1's ndcg@5: 0.269749
[3]	training's ndcg@5: 0.290604	valid_1's ndcg@5: 0.292316
[4]	training's ndcg@5: 0.296479	valid_1's ndcg@5: 0.2979
[5]	training's ndcg@5: 0.304303	valid_1's ndcg@5: 0.306088
[6]	training's ndcg@5: 0.314555	valid_1's ndcg@5: 0.315973
[7]	training's ndcg@5: 0.318118	valid_1's ndcg@5: 0.31978
[8]	training's ndcg@5: 0.323049	valid_1's ndcg@5: 0.324509
[9]	training's ndcg@5: 0.32746	valid_1's ndcg@5: 0.329454
[10]	training's ndcg@5: 0.330236	valid_1's ndcg@5: 0.331433
[11]	training's ndcg@5: 0.330921	valid_1's ndcg@5: 0.332461
[12]	training's ndcg@5: 0.333049	valid_1's ndcg@5: 0.334693
[13]	training's ndcg@5: 0.333591	valid_1's ndcg@5: 0.334663
[14]	training's ndcg@5: 0.334918	valid_1's ndcg@5: 0.335537
[15]	training's ndcg@5: 0.335433	valid_1's ndcg@5: 0.336086
[16]	training's ndcg@5: 0.336683	valid_1's ndcg@5: 0.337523
[17]	training's ndcg@5: 0.338086	valid_1's ndcg@5: 0.33

KeyboardInterrupt: 